# Get Data from Drive

In [ ]:
# %%bash

# cp -r Cars '/content/drive/MyDrive/Colab Stuff/Detection/Final_Cars/'
# cp -r datasets '/content/drive/MyDrive/Colab Stuff/Detection/Final_Cars/'
# cp -r '/content/faster_rcnn_resnet152_v1_640x640_coco17_tpu8' '/content/drive/MyDrive/Colab Stuff/Detection/Final_Cars/'
# cp -r models '/content/drive/MyDrive/Colab Stuff/Detection/Final_Cars/'
# cp -r functions_colab.py '/content/drive/MyDrive/Colab Stuff/Detection/Final_Cars/'
# cp -r generate_tfrecord.py '/content/drive/MyDrive/Colab Stuff/Detection/Final_Cars/'


In [ ]:
!cp -r /content/drive/MyDrive/Final_Cars/* .

# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import os

# Install object_detection

In [ ]:
%%bash

# pip install --ignore-installed --upgrade tensorflow==2.5.0
cd models/research 
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py . 
python -m pip install .
     

In [3]:
import tensorflow as tf

from object_detection.protos import pipeline_pb2
from google.protobuf import text_format


from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

2022-10-28 10:04:20.195992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-28 10:04:20.400588: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-28 10:04:21.222858: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/amr/anaconda3/lib/
2022-10-28 10:04:21.223111: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

# Initializations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from functions_colab import *
OD = Functions()

model = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz'
# model = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8.tar.gz'

oldmodelname, modelname = OD.download_model(model)

!mv 'faster_rcnn_resnet152_v1_640x640_coco17_tpu-8' 'faster_rcnn_resnet152_v1_640x640_coco17_tpu8'

# OD.moving('','faster_rcnn_resnet152_v1_640x640_coco17_tpu8')

# Paths 

In [ ]:
bbox_csv = pd.read_csv('Cars/data/Boxes.csv')
bbox_np = np.array(bbox_csv)
bbox_csv.tail()

In [4]:
modelname = 'faster_rcnn_resnet152_v1_640x640_coco17_tpu8'
train_img_dir = 'Cars/train/'
pipeline_config_dir = 'faster_rcnn_resnet152_v1_640x640_coco17_tpu8/pipeline.config'
Annotation_dir = 'Cars/Annotations/'
Annotation_file = 'Cars/Annotations/label_map.pbtxt'

# Labels

In [ ]:
labels = {'name': 'Car', 'id':1}

with open(Annotation_file, 'w') as f:
    f.write('item{\n')
    f.write('\tname:\'{}\'\n'.format(labels['name']))
    f.write('\tid:{}\n'.format(labels['id']))
    f.write('}\n')

# Creating XML Files

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
bbox_np = np.array(bbox_csv)
f1 = 'Cars/ref1.xml'
f2 = 'Cars/ref2.xml'
out = 'Cars/Annotations/xml/'
path = '/mnt/F/Machine_Learning/Projects/NewStuff/New Projects/Actual/Cars/train/'
ls = ['xmin', 'ymin', 'xmax', 'ymax', 'filename', 'path']


In [ ]:
for k in bbox_np:
    ls2_text_c = []
    xy = []
    name = k[0]
    exist = out + name[:-4] + '.xml'
    check = os.path.exists(exist)
    if not check:
        mytree = ET.parse(f1)
        root = mytree.getroot()
        for i in k[1:]:
            xy.append(round(i))
        xy.append(name)
        xy.append(path+name)
        for j,m in zip(ls,xy):
            for i in root.iter(j):
                i.text = str(m)
    else:
        names = [name, path+name]
        basic_values = []
        basic = ET.parse(exist)
        basic_root = basic.getroot()
        for i in basic_root.iter():
            if i.tag in ls[:4]:
                basic_values.append(i.text)
                
        mytree = ET.parse(f2)
        root = mytree.getroot()
        for l,m in zip(basic_values, ls[:4]):
            for i in root[6].iter(m):
                i.text = str(l)
        
        for op in k[1:]:
            ls2_text_c.append(round(op))
        
        for l,m in zip(ls2_text_c, ls[:4]):
            for i in root[7].iter(m):
                i.text = str(l)
                
        for l,m in zip(names, ls[4:]):
            for i in root.iter(m):
                i.text = l
        
        
    mytree.write(out+name[:-4]+'.xml')

# Generating Train and Test Records

In [ ]:
# ! rm Cars/Annotations/*.record

In [ ]:
!python generate_tfrecord.py -x 'Cars/train/'  -l 'Cars/Annotations/label_map.pbtxt' -o 'Cars/Annotations/train.record'
!python generate_tfrecord.py -x 'Cars/Validate/'  -l 'Cars/Annotations/label_map.pbtxt' -o 'Cars/Annotations/test.record'

# Model Config File

In [ ]:
!rm -r '/content/faster_rcnn_resnet152_v1_640x640_coco17_tpu8/SavedCP'
!mkdir '/content/faster_rcnn_resnet152_v1_640x640_coco17_tpu8/SavedCP'

In [ ]:
config_file = config_util.get_configs_from_pipeline_file(pipeline_config_dir)
config_file

In [ ]:
# ! cp pipeline.config '/content/faster_rcnn_resnet152_v1_800x1333_coco17_gpu8/'
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(pipeline_config_dir, "r") as f:                                                                                                                                                                                                                
    proto_str = f.read()                                                                                                                                                                                                    
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.faster_rcnn.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = modelname + '/checkpoint/ckpt-0'
# pipeline_config.train_config.fine_tune_checkpoint = '/content/faster_rcnn_resnet152_v1_800x1333_coco17_gpu8/SavedCP/ckpt-1'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= Annotation_file
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [Annotation_dir + 'train.record']
pipeline_config.eval_input_reader[0].label_map_path = Annotation_file
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [Annotation_dir + 'test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(pipeline_config_dir, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# Training

In [ ]:
print("""python models/research/object_detection/model_main_tf2.py --model_dir={}/SavedCP/ --pipeline_config_path={}/pipeline.config --alsologtostderr --num_train_steps=3000""".format(modelname, modelname))

In [ ]:
!python models/research/object_detection/model_main_tf2.py --model_dir=faster_rcnn_resnet152_v1_640x640_coco17_tpu8/SavedCP/ --pipeline_config_path=faster_rcnn_resnet152_v1_640x640_coco17_tpu8/pipeline.config --alsologtostderr --num_train_steps=3000

# Load Trained Model

In [5]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [6]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file('faster_rcnn_resnet152_v1_640x640_coco17_tpu8/pipeline.config')
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join('Saved Checkpoints/', 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

2022-10-28 10:04:35.132351: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 10:04:35.147545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 10:04:35.148018: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 10:04:35.148757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Making Predictions

## Functions

In [1]:
from google.colab.patches import cv2_imshow

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(Annotation_file)

In [8]:
def load_image(image):
    img1 = cv.imread(image)
    # img = cv.cvtColor(img1, cv.COLOR_BGR2RGB)
    img_v = np.array(img1)
    input_tensor = np.expand_dims(img_v.copy(),0)
    input_tensor = tf.convert_to_tensor(input_tensor, dtype=tf.float32)
    return img1, input_tensor

def visualize(img, detections):
    label_id_offset = 1
    viz_utils.visualize_boxes_and_labels_on_image_array(
                img,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.6,
                agnostic_mode=False)
    return img


def visualizer_func_helper(input_tensor):
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    return detections

## Image Testing

In [ ]:
path2test = 'Cars/test/'
test_files = os.listdir(path2test)

In [ ]:
for i in range(20):
    idx = np.random.randint(0,len(test_files))
    img, input_tensor = load_image(path2test + test_files[idx])
    detections = visualizer_func_helper(input_tensor)
    img2vis = visualize(img, detections)
    cv2_imshow(img2vis)

## Video Testing

In [ ]:
# %%bash
yt_vid = "https://www.youtube.com/watch?v=e_WBuBqS9h8"
# pip install yt-dlp
# yt-dlp -F yt_vid
yt-dlp -f 244 yt_vid

In [10]:
video_file = '30 Minutes of Cars Driving By in 2009 [e_WBuBqS9h8].webm'
capture = cv.VideoCapture(video_file)
fps = FPS().start()
frameTime = 10

while True:
    isTrue, frame = capture.read()
    img_f = np.array(frame)
    img_v = img_f.copy()
    input_tensor = np.expand_dims(img_v,0)
    input_tensor = tf.convert_to_tensor(input_tensor, dtype=tf.float32)
    detections = visualizer_func_helper(input_tensor)
    out = visualize(frame, detections)
    cv.imshow('vid',out)
#     cv.waitKey(1)
    if cv.waitKey(1) & 0xFF == ord('d'):
        break
    


capture.release()
cv.destroyAllWindows()

INFO:tensorflow:depth of additional conv before box predictor: 0
Instructions for updating:
Use ref() instead.


2022-10-28 10:05:15.288469: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2022-10-28 10:05:16.074777: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-10-28 10:05:16.093755: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-10-28 10:05:16.180669: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains